모든 내용은 [Keras Documentation](https://keras.io/) 를 참고하였음

# 목차
***

# Sequential Model 시작하기
***
<code class="keras">Sequential</code> 모델: 레이어의 선형 스택

## 간단한 예제

In [3]:
# 모델 생성/ 파라미터 넘기기
from keras.models import Sequential
from keras.layers import Dense, Activation

model = Sequential([
    Dense(32, input_shape=(784,)),
    Activation('relu'),
    Dense(10),
    Activation('softmax'),
])
# 직접 constructor에 파라미터를 넘김으로써 모델 생성 가능
# Dense: input wire의 dimension을 설정
# Acitivation: 레이어의 activation함수를 설정

Using TensorFlow backend.


In [ ]:
# model.add 메소드를 통해 추가할 수도 있다. 위 코드와 strictly equivalent 하다고 한다.
model = Sequential()
model.add(Dense(32, input_dim=784))
model.add(Activation('relu'))

## Input Shape 지정하기
모델은 input shape을 알아야 한다. 따라서 모델의 첫 레이어(첫 번째만, 그 이후는 자동적으로 추론함)는 input shape에 대한 정보를 받아야 한다. 가능한 방법은 아래와 같다.

1. 첫 레이어에 <code class="keras">input_shape</code> 인수(argument)를 튜플 형태로 지정
2. 2D 레이어에 대해(e.g. <code class="keras">Dense</code>) <code class="keras">input_dim</code> 인수를, 3D 입력 레이어에 대해 <code class="keras">input_dim</code>과 <code class="keras">input_length</code> 인수를 지정
3. 고정된 배치 사이즈를 지정해야 하는 경우, <code class="keras">batch_size</code> 인수를 레이어에 넘길 수 있다. 예를 들어, <code class="keras">batch_size=32</code>와 <code class="keras">input_shape=(6,8)</code> 인수를 레이어에 넘기는 경우 배치 shape은 <code class="keras">(32,6,8)</code>이 된다.

In [4]:
# 1번 예시
model = Sequential()
model.add(Dense(32, input_shape=(784,)))

In [6]:
#2번 예시
model = Sequential()
model.add(Dense(32, input_dim=784))

## 모델 컴파일
모델을 training하기 전에 compile 메소드를 통해 컴파일 해야 한다. compile 메소드는 3 가지 인수를 받는다.
* optimizer: *optimizer*의 이름을 문자열로 넘기거나 <code class="keras">Optimizer</code> 클래스의 인스턴스를 넘겨도 된다. [optimizer doc](https://keras.io/optimizers) 참고
* loss function: 모델이 최소화하는 함수. 문자열을 넘기거나 objective function을 넘길 수 있다. [losses doc](https://keras.io/losses) 참고
* a list of metrics: 어떤 종류의 분류 문제든 <code class="keras">metrics=['accuracy']</code>을 사용할 수 있다. 이미 존재하는 metric의 문자열 또는 직접 metric함수를 만들어 넘길 수 있다.

In [4]:
# For a multi-class classification problem
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [5]:
# For a binary classification problem
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
# For a mean squared error regression problem
model.compile(optimizer='rmsprop',
              loss='mse')

In [6]:
# For custom metrics
import keras.backend as K

def mean_pred(y_true, y_pred):
    return K.mean(y_pred)

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy', mean_pred])

## 모델 훈련/학습(Training)
Numpy array로 입력 데이터와 레이블을 만들어 낸 후 <code class="keras">fit</code> 메소드를 사용한다.

In [9]:
# For a single-input model with 2 classes (binary classification):

model = Sequential()
model.add(Dense(32, activation='relu', input_dim=100))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Generate dummy data
import numpy as np
data = np.random.random((1000, 100))
labels = np.random.randint(2, size=(1000, 1))

# Train the model, iterating on the data in batches of 32 samples
model.fit(data, labels, epochs=10, batch_size=32, verbose=0)

In [12]:
from keras.utils import to_categorical
# For a single-input model with 10 classes (categorical classification):

model = Sequential()
model.add(Dense(32, activation='relu', input_dim=100))
model.add(Dense(10, activation='softmax'))
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Generate dummy data
import numpy as np
data = np.random.random((1000, 100))
labels = np.random.randint(10, size=(1000, 1))

# Convert labels to categorical one-hot encoding
one_hot_labels = to_categorical(labels, num_classes=10)

# Train the model, iterating on the data in batches of 32 samples
model.fit(data, one_hot_labels, epochs=10, batch_size=32, verbose=0)

# Keras functional API 시작하기
<code class="keras">Sequential</code> 모델에 익숙하다고 가정한다.

## 첫 예제

In [16]:
from keras.layers import Input, Dense
from keras.models import Model

# This returns a tensor
inputs = Input(shape=(784,))

# a layer instance is callable on a tensor, and returns a tensor
x = Dense(64, activation='relu')(inputs)
x = Dense(64, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)

# This creates a model that includes
# the Input layer and three Dense layers
model = Model(inputs=inputs, outputs=predictions)
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(data, labels)  # starts training, data가 없어서 실행은 안됨

ValueError: Error when checking input: expected input_4 to have shape (784,) but got array with shape (100,)

* layer 인스턴스는 callable하며 tensor를 리턴한다.
* Input tensor와 output tensor는 <code class="keras">Model</code>을 정의하는데 쓰일 수 있다.
* 이런 모델은 <code class="keras">Sequential</code>과 거의 동일하게 동작한다.

## 모든 모델은 callable하다 (레이어도 마찬가지)
모델을 레이어처럼 취급 가능하다. 단, 모델을 호출한다는 것은 모델의 _구조_뿐 아니라 파라미터의 _가중치(weight)_도 그대로 이용한다는 것을 의미한다.

In [19]:
x = Input(shape=(784,))
# This works, and returns the 10-way softmax we defined above.
y = model(x)

이와 같은 특성을 이용해 이미지 분류 모델을 비디오 분류 모델로 쉽게 바꾸는 것도 가능하다.

In [20]:
from keras.layers import TimeDistributed

# Input tensor for sequences of 20 timesteps,
# each containing a 784-dimensional vector
input_sequences = Input(shape=(20, 784))

# This applies our previous model to every timestep in the input sequences.
# the output of the previous model was a 10-way softmax,
# so the output of the layer below will be a sequence of 20 vectors of size 10.
processed_sequences = TimeDistributed(model)(input_sequences)

## 다중 입력(Multi-input)과 다중 출력(Multi-output) 모델
예를 들어 어떤 뉴스 헤드라인에 리트윗과 좋아요를 얼마나 받게 될지 예측하는 모델을 만든다고 한다. 주요 입력(main_input)은 단어들의 시퀀스, 즉 헤드라인 그 자체와  뉴스가 포스팅된 시간 등의 추가적인 입력(aux_input)이 있을 것이다. 이 모델은 2개의 loss function을 통해 supervised될 것이다. 주요 loss function을 모델 초기 단계에 사용하는 것은 좋은 정규화(regularization) 메커니즘이다.

<img src="multi-input-multi-output-graph.png" alt="multi-input-multi-output" style="width: 400px;"/>

위 그림대로 모델을 구현해보자. 위 그림에서 괄호 안은 클래스 이름을 의미한다.

In [22]:
# 구현:
# main_input(InputLayer)
# embedding_1(Embedding)
# lstm_1(LSTM)
from keras.layers import Input, Embedding, LSTM, Dense
from keras.models import Model

# Headline input: meant to receive sequences of 100 integers, between 1 and 10000.
# Note that we can name any layer by passing it a "name" argument.
main_input = Input(shape=(100,), dtype='int32', name='main_input')

# This embedding layer will encode the input sequence
# into a sequence of dense 512-dimensional vectors.
x = Embedding(output_dim=512, input_dim=10000, input_length=100)(main_input)

# A LSTM will transform the vector sequence into a single vector,
# containing information about the entire sequence
lstm_out = LSTM(32)(x)

In [24]:
# 구현:
# aux_output(Dense)
auxiliary_output = Dense(1, activation='sigmoid', name='aux_output')(lstm_out)

In [26]:
# 구현: 나머지
auxiliary_input = Input(shape=(5,), name='aux_input')
x = keras.layers.concatenate([lstm_out, auxiliary_input])

# We stack a deep densely-connected network on top
x = Dense(64, activation='relu')(x)
x = Dense(64, activation='relu')(x)
x = Dense(64, activation='relu')(x)

# And finally we add the main logistic regression layer
main_output = Dense(1, activation='sigmoid', name='main_output')(x)

In [30]:
# main_input, auxiliary_input / main_output, auxiliary_output ==> 모델 정의
model = Model(inputs=[main_input, auxiliary_input], 
                       outputs=[main_output, auxiliary_output])

In [31]:
# 모델 컴파일
model.compile(optimizer='rmsprop', loss='binary_crossentropy',
              loss_weights=[1., 0.2])
# 이름 지정하여 인수 넘기는 방법
model.compile(optimizer='rmsprop',
              loss={'main_output': 'binary_crossentropy', 'aux_output': 'binary_crossentropy'},
              loss_weights={'main_output': 1., 'aux_output': 0.2})

In [33]:
# 모델 Training
model.fit([headline_data, additional_data], [labels, labels],
          epochs=50, batch_size=32)
# 이름 지정하여 인수 넘기는 방법
model.fit({'main_input': headline_data, 'aux_input': additional_data},
          {'main_output': labels, 'aux_output': labels},
          epochs=50, batch_size=32)

NameError: name 'headline_data' is not defined

## Shared Layers
두 개의 트윗의 작성자가 같은지 알아보는 모델을 만든다고 하자. 이를 하기 위해서는 두 트윗을 두 개의 벡터로 만든 후 두 벡터를 붙여(concatenate) 로지스틱 회귀를 더한다. 이 모델의 결과물은 두 트윗의 작성자가 같을 확률이다. 모델은 positive tweet pairs과 negative tweet pairs 모두를 학습한다.

문제가 대칭이므로 트윗의 처음부분(positive tweet pairs)을 encode하는 방법이 그대로 뒷 부분(negative tweet pairs)에 사용될 수 있다. 여기서는 LSTM 레이어를 사용한다.

한 트윗에 대한 입력은 (280, 256)차원의 바이너리 행렬이다. 즉 280은 트위터 글자수 길이를 의미하며 256은 가장 많이 쓰이는 알파벳 256개의 포함 여부( [0/1] )이다.

In [35]:
import keras
from keras.layers import Input, LSTM, Dense
from keras.models import Model

tweet_a = Input(shape=(280, 256))
tweet_b = Input(shape=(280, 256))

레이러를 공유하기 위해서는 단순히 레이어 인스턴스를 한 번 만들고 각 입력 데이터에 대해 여러번 호출하면 된다.

In [36]:
# This layer can take as input a matrix
# and will return a vector of size 64
# 인스턴스 생성은 한 번
shared_lstm = LSTM(64)

# When we reuse the same layer instance
# multiple times, the weights of the layer
# are also being reused
# (it is effectively *the same* layer)
encoded_a = shared_lstm(tweet_a)
encoded_b = shared_lstm(tweet_b)

# We can then concatenate the two vectors:
merged_vector = keras.layers.concatenate([encoded_a, encoded_b], axis= -1)

# And add a logistic regression on top
predictions = Dense(1, activation='sigmoid')(merged_vector)

# We define a trainable model linking the
# tweet inputs to the predictions
model = Model(inputs=[tweet_a, tweet_b], outputs=predictions)

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.fit([data_a, data_b], labels, epochs=10)

NameError: name 'data_a' is not defined

## The concept of layer "node"
레이어 호출은 새로운 텐서(i.e. 레이어의 새로운 출력)를 생성하여 "node"를 그 레이어에 더하는 것을 의미한다. 다시 말해, 입력 텐서와 출력 텐서를 연결하는 것이다. 같은 레이어를 여러 번 호출하는 것은 그 레이어가 0부터 인덱싱되는 여러 노드를 소유하고 있다는 것을 의미한다.

레이어가 하나의 입력만 연결되어있다면 혼동될 여지 없이 <code class="keras">.output</code>은 레이어의 하나의 출력을 리턴할 것이다.

In [39]:
a = Input(shape=(280, 256))

lstm = LSTM(32)
encoded_a = lstm(a)

assert lstm.output == encoded_a

In [42]:
# ERROR: input이 여러 개면 indexing이 필요
a = Input(shape=(280, 256))
b = Input(shape=(280, 256))

lstm = LSTM(32)
encoded_a = lstm(a)
encoded_b = lstm(b)

lstm.output

AttributeError: Layer lstm_8 has multiple inbound nodes, hence the notion of "layer output" is ill-defined. Use `get_output_at(node_index)` instead.

In [43]:
# 인덱싱 
assert lstm.get_output_at(0) == encoded_a
assert lstm.get_output_at(1) == encoded_b

<code class="keras">input_shape</code>과 <code class="keras">output_shape</code> 속성도 <code class="keras">output</code> 속성과 동일하다.

In [45]:
from keras.layers import Input, Conv2D
a = Input(shape=(32, 32, 3))
b = Input(shape=(64, 64, 3))

conv = Conv2D(16, (3, 3), padding='same')
conved_a = conv(a)

# Only one input so far, the following will work:
assert conv.input_shape == (None, 32, 32, 3)

conved_b = conv(b)
# now the `.input_shape` property wouldn't work, but this does:
assert conv.get_input_shape_at(0) == (None, 32, 32, 3)
assert conv.get_input_shape_at(1) == (None, 64, 64, 3)